In [1]:
%run init.ipynb

matchzoo version 1.1.1
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
/Users/yyang/.matchzoo/datasets/wiki_qa/WikiQACorpus
/Users/yyang/.matchzoo/datasets/wiki_qa/WikiQACorpus
/Users/yyang/.matchzoo/datasets/wiki_qa/WikiQACorpus
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.models.ConvKNRM.get_default_preprocessor()

In [7]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 6064.79it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:08<00:00, 2107.10it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418412/418412 [00:00<00:00, 2781638.93it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9381.11it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4368.44it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 313268.83it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 3402.05it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [8]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x10cc72790>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x10b99a750>,
 'vocab_size': 30059,
 'embedding_input_dim': 30059}

In [9]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [10]:
glove_embedding

In [6]:
#batchsize can be modified here
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=1
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [7]:
padding_callback = mz.models.ConvKNRM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    #batch_size=20,
    stage='train',
    #resample=True,
    #sort=False,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    #batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [8]:
model = mz.models.ConvKNRM()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['filters'] = 128 
model.params['conv_activation_func'] = 'tanh' 
model.params['max_ngram'] = 3
model.params['use_crossmatch'] = True 
model.params['kernel_num'] = 11
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

ConvKNRM(
  (embedding): Embedding(30059, 300, padding_idx=0)
  (q_convs): ModuleList(
    (0): Sequential(
      (0): ConstantPad1d(padding=(0, 0), value=0)
      (1): Conv1d(300, 128, kernel_size=(1,), stride=(1,))
      (2): Tanh()
    )
    (1): Sequential(
      (0): ConstantPad1d(padding=(0, 1), value=0)
      (1): Conv1d(300, 128, kernel_size=(2,), stride=(1,))
      (2): Tanh()
    )
    (2): Sequential(
      (0): ConstantPad1d(padding=(0, 2), value=0)
      (1): Conv1d(300, 128, kernel_size=(3,), stride=(1,))
      (2): Tanh()
    )
  )
  (d_convs): ModuleList(
    (0): Sequential(
      (0): ConstantPad1d(padding=(0, 0), value=0)
      (1): Conv1d(300, 128, kernel_size=(1,), stride=(1,))
      (2): Tanh()
    )
    (1): Sequential(
      (0): ConstantPad1d(padding=(0, 1), value=0)
      (1): Conv1d(300, 128, kernel_size=(2,), stride=(1,))
      (2): Tanh()
    )
    (2): Sequential(
      (0): ConstantPad1d(padding=(0, 2), value=0)
      (1): Conv1d(300, 128, kernel_size=(3,

In [9]:
optimizer = torch.optim.Adadelta(model.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10,
    scheduler=scheduler,
    clip_norm=10
)

In [10]:
#rankhingeloss
trainer.run()

/Users/yyang/opt/anaconda3/envs/conda_env/lib/python3.7/site-packages/matchzoo_py-1.1.1-py3.7.egg/matchzoo/data_pack/data_pack.py:151: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x[key] = np.array(val)


[Iter-160 Loss-0.452]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4881 - normalized_discounted_cumulative_gain@5(0.0): 0.5628 - mean_average_precision(0.0): 0.5064



[Iter-320 Loss-0.077]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5046 - normalized_discounted_cumulative_gain@5(0.0): 0.5905 - mean_average_precision(0.0): 0.5366



[Iter-480 Loss-0.027]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5219 - normalized_discounted_cumulative_gain@5(0.0): 0.596 - mean_average_precision(0.0): 0.549



[Iter-640 Loss-0.008]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5087 - normalized_discounted_cumulative_gain@5(0.0): 0.5874 - mean_average_precision(0.0): 0.5336



...

[Iter-960 Loss-0.001]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5179 - normalized_discounted_cumulative_gain@5(0.0): 0.5785 - mean_average_precision(0.0): 0.5227



[Iter-1120 Loss-0.001]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5035 - normalized_discounted_cumulative_gain@5(0.0): 0.5781 - mean_average_precision(0.0): 0.5251



[Iter-1280 Loss-0.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5097 - normalized_discounted_cumulative_gain@5(0.0): 0.5811 - mean_average_precision(0.0): 0.5327



[Iter-1440 Loss-0.001]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5015 - normalized_discounted_cumulative_gain@5(0.0): 0.5812 - mean_average_precision(0.0): 0.5285



[Iter-1600 Loss-0.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4917 - normalized_discounted_cumulative_gain@5(0.0): 0.5668 - mean_average_precision(0.0): 0.5096

Cost time: 969.8109769821167s


In [14]:
#rankcrossentropyloss
trainer.run()

/Users/yyang/opt/anaconda3/envs/conda_env/lib/python3.7/site-packages/matchzoo_py-1.1.1-py3.7.egg/matchzoo/data_pack/data_pack.py:151: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x[key] = np.array(val)


[Iter-160 Loss-0.344]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5663 - normalized_discounted_cumulative_gain@5(0.0): 0.6253 - mean_average_precision(0.0): 0.5716



[Iter-320 Loss-0.038]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4864 - normalized_discounted_cumulative_gain@5(0.0): 0.5669 - mean_average_precision(0.0): 0.5156



[Iter-480 Loss-0.006]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5672 - normalized_discounted_cumulative_gain@5(0.0): 0.6131 - mean_average_precision(0.0): 0.5705



[Iter-640 Loss-0.004]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5494 - normalized_discounted_cumulative_gain@5(0.0): 0.6086 - mean_average_precision(0.0): 0.5654



...

[Iter-960 Loss-0.001]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5129 - normalized_discounted_cumulative_gain@5(0.0): 0.5907 - mean_average_precision(0.0): 0.5395



[Iter-1120 Loss-0.001]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5476 - normalized_discounted_cumulative_gain@5(0.0): 0.6134 - mean_average_precision(0.0): 0.5683



[Iter-1280 Loss-0.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5377 - normalized_discounted_cumulative_gain@5(0.0): 0.6013 - mean_average_precision(0.0): 0.5523



[Iter-1440 Loss-0.001]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5544 - normalized_discounted_cumulative_gain@5(0.0): 0.6102 - mean_average_precision(0.0): 0.5721



[Iter-1600 Loss-0.001]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5203 - normalized_discounted_cumulative_gain@5(0.0): 0.5862 - mean_average_precision(0.0): 0.54

Cost time: 990.1388800144196s


In [16]:
for i in range(11):
    print((1+2. * i)/(11 - 1)-1)

-0.9
-0.7
-0.5
-0.30000000000000004
-0.09999999999999998
0.10000000000000009
0.30000000000000004
0.5
0.7
0.8999999999999999
1.1
